In [ ]:
import numpy as np
import pandas as pd
import sklearn
import copy as cp
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import classification_report
import seaborn as sns
from typing import Tuple
from sklearn import svm, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
df = pd.read_csv('/content/genres_v2.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,...,0VSXnJqQkwuH2ei1nOQ1nu,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,...,4jCeguq9rMTlbMmPHuO7S3,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN


In [ ]:
genre = df.genre.tolist()
words = ['Dark Trap', 'Underground Rap', 'Trap Metal', 'Emo','Rap','RnB','Pop','Hiphop','techhouse','techno','trance','psytrance','trap','dnb','hardstyle']


In [ ]:
l1 = []
count = 0
for item in genre:
    if item not in l1:
        count += 1
        l1.append(item)

print("There are:", count, "different genres")
print(pd.value_counts(np.array(genre)))


There are: 15 different genres
Underground Rap    5875
Dark Trap          4578
Hiphop             3028
trance             2999
trap               2987
techhouse          2975
dnb                2966
psytrance          2961
techno             2956
hardstyle          2936
RnB                2099
Trap Metal         1956
Rap                1848
Emo                1680
Pop                 461
dtype: int64


In [ ]:
genre=genre=pd.get_dummies(genre)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['genre_encoded'] = le.fit_transform(df['genre'])
df.head() 



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,uri,track_href,analysis_url,duration_ms,time_signature,genre,song_name,Unnamed: 0,title,genre_encoded
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,https://api.spotify.com/v1/tracks/2Vc6NJ9PW9gD...,https://api.spotify.com/v1/audio-analysis/2Vc6...,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN,0
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,https://api.spotify.com/v1/tracks/7pgJBLVz5Vmn...,https://api.spotify.com/v1/audio-analysis/7pgJ...,224427,4,Dark Trap,Pathology,NaN,NaN,0
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,spotify:track:0vSWgAlfpye0WCGeNmuNhy,https://api.spotify.com/v1/tracks/0vSWgAlfpye0...,https://api.spotify.com/v1/audio-analysis/0vSW...,98821,4,Dark Trap,Symbiote,NaN,NaN,0
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,...,spotify:track:0VSXnJqQkwuH2ei1nOQ1nu,https://api.spotify.com/v1/tracks/0VSXnJqQkwuH...,https://api.spotify.com/v1/audio-analysis/0VSX...,123661,3,Dark Trap,ProductOfDrugs (Prod. The Virus and Antidote),NaN,NaN,0
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,...,spotify:track:4jCeguq9rMTlbMmPHuO7S3,https://api.spotify.com/v1/tracks/4jCeguq9rMTl...,https://api.spotify.com/v1/audio-analysis/4jCe...,123298,4,Dark Trap,Venom,NaN,NaN,0


In [ ]:


X = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',  'duration_ms',
       'time_signature']]

y = df[['genre_encoded']]

features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms' ]
X.head()


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,124539,4
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,224427,4
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,98821,4
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,123661,3
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,123298,4


In [ ]:
y.head()

,genre_encoded
0,0
1,0
2,0
3,0
4,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train ,y_test = train_test_split(X,y,random_state =0)

In [ ]:

accuracy = [['model','accuracy']]


In [ ]:

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)

cm = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.3028268885317198


In [ ]:
import numpy as np
import pandas as pd
import pretty_confusion_matrix

yhat = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)


0.3028268885317198


In [ ]:


from sklearn.metrics import accuracy_score

accuracy.append(['LogisticRegression', accuracy_score(y_test , yhat, normalize =True)])
accuracy_score(y_test , yhat, normalize =True)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.18      0.01      0.01      1160
           1       0.00      0.00      0.00       391
           2       0.00      0.00      0.00       765
           3       0.00      0.00      0.00       104
           4       0.00      0.00      0.00       448
           5       0.00      0.00      0.00       549
           6       0.00      0.00      0.00       506
           7       0.23      0.84      0.36      1472
           8       0.16      0.51      0.24       735
           9       0.00      0.00      0.00       722
          10       0.36      0.90      0.51       762
          11       0.19      0.04      0.06       767
          12       0.64      0.01      0.02       710
          13       0.10      0.10      0.10       735
          14       0.00      0.00      0.00       751

    accuracy                           0.23     10577
   macro avg       0.12      0.16      0.09     10577
weighted avg       0.15   

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth = 10 , random_state  = 0)
model.fit(X_train,y_train)



<ipython-input-203-fe379266c10c>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y_train)


KeyboardInterrupt: ignored

In [ ]:
y_train.info()
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31728 entries, 34418 to 2732
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   genre_encoded  31728 non-null  int64
dtypes: int64(1)
memory usage: 495.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10577 entries, 19070 to 6055
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   genre_encoded  10577 non-null  int64
dtypes: int64(1)
memory usage: 165.3 KB


In [ ]:
accuracy.append(['Random Forest',accuracy_score(y_test,yhat)])
accuracy_score(y_test,yhat)



0.22870379124515458

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))




/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1160
           1       0.00      0.00      0.00       391
           2       0.00      0.00      0.00       765
           3       0.00      0.00      0.00       104
           4       0.00      0.00      0.00       448
           5       0.00      0.00      0.00       549
           6       0.00      0.00      0.00       506
           7       0.30      0.01      0.01      1472
           8       0.00      0.00      0.00       735
           9       0.07      1.00      0.13       722
          10       0.00      0.00      0.00       762
          11       0.00      0.00      0.00       767
          12       0.00      0.00      0.00       710
          13       0.00      0.00      0.00       735
          14       0.00      0.00      0.00       751

    accuracy                           0.07     10577
   macro avg       0.02      0.07      0.01     10577
weighted avg       0.05   

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

In [ ]:
accuracy_score(y_test,yhat)
cm = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)
print(acc_score)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

knn.score(X_train, y_train)

color = 'white'
matrix = plot_confusion_matrix(knn, X_test, y_pred, cmap=plt.cm.Blues)
matrix.ax_.set_title('Confusion Matrix', color=color)
plt.xlabel('Predicted Label', color=color)
plt.ylabel('True Label', color=color)
plt.gcf().axes[0].tick_params(colors=color)
plt.gcf().axes[1].tick_params(colors=color)
plt.show()

In [ ]:
 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.18      0.27      0.21      1160
           1       0.11      0.14      0.12       391
           2       0.17      0.15      0.16       765
           3       0.00      0.00      0.00       104
           4       0.09      0.05      0.07       448
           5       0.16      0.09      0.12       549
           6       0.20      0.15      0.17       506
           7       0.26      0.23      0.24      1472
           8       0.46      0.53      0.49       735
           9       0.32      0.36      0.34       722
          10       0.53      0.68      0.60       762
          11       0.41      0.38      0.40       767
          12       0.41      0.38      0.39       710
          13       0.26      0.20      0.22       735
          14       0.45      0.48      0.46       751

    accuracy                           0.30     10577
   macro avg       0.27      0.27      0.27     10577
weighted avg       0.29   

In [ ]:
accuracy.append(['SVM', accuracy_score(y_test,yhat)])
accuracy_score(y_test,yhat)

print(classification_report(y_test, yhat))


              precision    recall  f1-score   support

           0       0.18      0.01      0.01      1160
           1       0.00      0.00      0.00       391
           2       0.00      0.00      0.00       765
           3       0.00      0.00      0.00       104
           4       0.00      0.00      0.00       448
           5       0.00      0.00      0.00       549
           6       0.00      0.00      0.00       506
           7       0.23      0.84      0.36      1472
           8       0.16      0.51      0.24       735
           9       0.00      0.00      0.00       722
          10       0.36      0.90      0.51       762
          11       0.19      0.04      0.06       767
          12       0.64      0.01      0.02       710
          13       0.10      0.10      0.10       735
          14       0.00      0.00      0.00       751

    accuracy                           0.23     10577
   macro avg       0.12      0.16      0.09     10577
weighted avg       0.15   

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
 genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42305 entries, 0 to 42304
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Dark Trap        42305 non-null  uint8
 1   Emo              42305 non-null  uint8
 2   Hiphop           42305 non-null  uint8
 3   Pop              42305 non-null  uint8
 4   Rap              42305 non-null  uint8
 5   RnB              42305 non-null  uint8
 6   Trap Metal       42305 non-null  uint8
 7   Underground Rap  42305 non-null  uint8
 8   dnb              42305 non-null  uint8
 9   hardstyle        42305 non-null  uint8
 10  psytrance        42305 non-null  uint8
 11  techhouse        42305 non-null  uint8
 12  techno           42305 non-null  uint8
 13  trance           42305 non-null  uint8
 14  trap             42305 non-null  uint8
dtypes: uint8(15)
memory usage: 619.8 KB


In [ ]:
y=genre

In [ ]:
y.head(5)

In [ ]:
from sklearn import preprocessing
from sklearn import pipeline

poly = preprocessing.PolynomialFeatures(degree=30, include_bias=False)
scaler = preprocessing.StandardScaler()

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

enc


In [ ]:
y = enc.fit_transform(df[["genre"]])



              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1160
           1       0.00      0.00      0.00       391
           2       0.00      0.00      0.00       765
           3       0.00      0.00      0.00       104
           4       0.00      0.00      0.00       448
           5       0.00      0.00      0.00       549
           6       0.00      0.00      0.00       506
           7       0.30      0.01      0.01      1472
           8       0.00      0.00      0.00       735
           9       0.07      1.00      0.13       722
          10       0.00      0.00      0.00       762
          11       0.00      0.00      0.00       767
          12       0.00      0.00      0.00       710
          13       0.00      0.00      0.00       735
          14       0.00      0.00      0.00       751

    accuracy                           0.07     10577
   macro avg       0.02      0.07      0.01     10577
weighted avg       0.05   

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pd.DataFrame(y)

,0
0,"(0, 0)\t1.0"
1,"(0, 0)\t1.0"
2,"(0, 0)\t1.0"
3,"(0, 0)\t1.0"
4,"(0, 0)\t1.0"
...,...
42300,"(0, 9)\t1.0"
42301,"(0, 9)\t1.0"
42302,"(0, 9)\t1.0"
42303,"(0, 9)\t1.0"


In [ ]:
drop_data = df.drop(columns=['id','uri','track_href','analysis_url','song_name','title','Unnamed: 0','genre','type'])


In [ ]:
drop_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre_encoded
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,124539,4,0
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,224427,4,0
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,98821,4,0
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,123661,3,0
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,123298,4,0


In [ ]:
from sklearn.model_selection import KFold
X = drop_data
kf = KFold(n_splits=5)
kf.get_n_splits(X)

5